In [6]:
# imports
from utils.pointcloud import BBox, grid_subsample_simple
from data.fwf_dataset import FwfDataset
from models.fgf import FGFeatNetwork
from omegaconf import OmegaConf


import numpy as np
import json
import os


# build config
cfg = OmegaConf.load("./config/default.yaml")
cfg = OmegaConf.merge(cfg, OmegaConf.load(cfg.data.split))
with open(os.path.join(cfg.data.dataset_root, 'class_dict.json'),'r') as f:
    cfg = OmegaConf.merge(cfg, OmegaConf.create({'data':{'label_schema':json.load(f)}}))


print(OmegaConf.to_yaml(cfg))






general:
  device: cuda
  batch_size: 128
  weight_decay: 0.0001
  max_epochs: 50
data:
  dataset_root: D:\Projekte\GIA_220412_PCS\02_Datasets\FullWaveForm\full_waveform_software\FullWaveformParse_mass\output\FWF_Aachen_labeled\
  split: ./config/default_split.yaml
  preprocessing:
    _transformsTraining_:
      xyz:
        TransZRotation: {}
        TransScaling: {}
        TransGaussianNoise:
          mean: 0
          std: 0.002
      rgb:
        TransGaussianNoise:
          mean: 0
          std: 0.02
        TransFeatureDropout:
          dropout_prob: 0.1
      wfm:
        TransSignalScaling: {}
      normals:
        TransGaussianNoise:
          mean: 0
          std: 0.05
      incAngles:
        TransGaussianNoise:
          mean: 0
          std: 0.05
        TransStandardize:
          mean: 2.07
          std: 0.38
      distanceFromScanner:
        TransGaussianNoise:
          mean: 12.13
          std: 12.2
    _transformsValidation_:
      xyz:
        TransZRota

In [7]:
train_ds = FwfDataset(cfg, cfg.data.preprocessing._transformsTraining_, cfg.data._trainProjects_)
val_ds = FwfDataset(cfg, cfg.data.preprocessing._transformsValidation_, cfg.data._valProjects_)

# rest computation
for ds in [train_ds, val_ds]:
    ds.compute_neibors_knn(k=cfg.data.num_neib_normalsComputation)
    ds.compute_normals_knn()
    ds.compute_incAngles()
    ds.compute_neibors_knn(k=cfg.data.num_neib_featureExtraction)
    

Loading '2024-04-05_FW_Westbahnhof_02.FwfProj'; Bounding box IDs = default
Loading '2023-08-28_FW_EingangBauing.FwfProj'; Bounding box IDs = default
Computing neibors for '2024-04-05_FW_Westbahnhof_02.FwfProj::defaultBbox' @ k=20
Computing normals for '2024-04-05_FW_Westbahnhof_02.FwfProj::defaultBbox' @ k=20
Computing incidence angles for '2024-04-05_FW_Westbahnhof_02.FwfProj::defaultBbox'
Computing neibors for '2024-04-05_FW_Westbahnhof_02.FwfProj::defaultBbox' @ k=128
Computing neibors for '2023-08-28_FW_EingangBauing.FwfProj::defaultBbox' @ k=20
Computing normals for '2023-08-28_FW_EingangBauing.FwfProj::defaultBbox' @ k=20
Computing incidence angles for '2023-08-28_FW_EingangBauing.FwfProj::defaultBbox'
Computing neibors for '2023-08-28_FW_EingangBauing.FwfProj::defaultBbox' @ k=128


In [13]:
from torch.utils.data import DataLoader

# create model
model = FGFeatNetwork(cfg=cfg,
    num_input_feats = train_ds[0]['features_neibors'].shape[-1],
    ).to(device=cfg.general.device)

# create dataloader
train_dl = DataLoader(train_ds, batch_size=cfg.general.batch_size)
val_dl = DataLoader(val_ds, batch_size=cfg.general.batch_size)

In [14]:
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
import torch
from tqdm import tqdm
from utils.metrics import get_multilevel_metrics, print_metrics


criterion = CrossEntropyLoss()

optim = Adam(params=model.parameters(), weight_decay=cfg.general.weight_decay)

for epoch in range(cfg.general.max_epochs):
    print(f"Epoch-{epoch:03}")



    # train
    epoch_train_loss = []
    epoch_predictions = {level:list() for level in cfg.data.label_names}
    epoch_labels = {level:list() for level in cfg.data.label_names}
    model.train()
    for i, batch in enumerate(tqdm(train_dl, desc=f"{'Training':<15}", leave=True)):
        optim.zero_grad()
        # put batch on device
        for k, v in batch.items():
            batch[k] = v.to(device=cfg.general.device)

        # forward pass
        out = model(batch)
        
        # agregate loss on all levels
        loss = torch.tensor(0.).to(device=cfg.general.device)
        for k in out.keys():
            output = out[k]
            gt = batch[k] # type:ignore
            loss += criterion(output, gt)
        
        loss.backward()
        optim.step()


        # aggregate values for metric calculation
        epoch_train_loss.append(loss.item())
        for level in cfg.data.label_names:
            epoch_predictions[level].append(torch.argmax(out[level],dim=1))
            epoch_labels[level].append(batch[level])

    # calculate_metrics
    for level in cfg.data.label_names:
        epoch_predictions[level] = torch.cat(epoch_predictions[level])
        epoch_labels[level] = torch.cat(epoch_labels[level])

    metrics = get_multilevel_metrics(epoch_predictions, epoch_labels, cfg)
    print_metrics(metrics, cfg)

    
    # validate  
    epoch_predictions = {level:list() for level in cfg.data.label_names}
    epoch_labels = {level:list() for level in cfg.data.label_names}  
    epoch_val_loss = []
    model.eval()
    with torch.no_grad():
        for i,batch in enumerate(tqdm(val_dl, desc=f"{'Validation':<15}", leave=True)):
            # put batch on device
            for k, v in batch.items():
                batch[k] = v.to(device=cfg.general.device)

            # forward pass
            out = model(batch)
            
            # agregate loss on all levels
            loss = torch.tensor(0.).to(device=cfg.general.device)
            for k in out.keys():
                output = out[k]
                gt = batch[k] # type:ignore
                loss += criterion(output, gt)
            
            # aggregate values for metric calculation
            epoch_val_loss.append(loss.item())
            for level in cfg.data.label_names:
                epoch_predictions[level].append(torch.argmax(out[level],dim=1))
                epoch_labels[level].append(batch[level])

        # calculate_metrics
        for level in cfg.data.label_names:
            epoch_predictions[level] = torch.cat(epoch_predictions[level])
            epoch_labels[level] = torch.cat(epoch_labels[level])

        metrics = get_multilevel_metrics(epoch_predictions, epoch_labels, cfg)
        print_metrics(metrics, cfg)

    print(f"L_train:{np.mean(epoch_train_loss):.4f} L_val:{np.mean(epoch_val_loss):.4f}")
    print('\n')
    break
    

Epoch-000


Training       : 100%|██████████| 64/64 [00:05<00:00, 12.58it/s]


                oA      mIoU        mP        mR      mF1     mcAcc
labels_0  0.874434  0.292361  0.301548  0.328837  0.31278  0.328837


Validation     :  18%|█▊        | 627/3566 [00:37<02:55, 16.75it/s]


KeyboardInterrupt: 

In [6]:
metrics

NameError: name 'metrics' is not defined

In [ ]:
out['labels_0'].shape

torch.Size([109, 3])

In [ ]:
preds = {k:torch.argmax(v,dim=1) for k,v in out.items()}
preds

{'labels_0': tensor([1, 0, 2, 1, 2, 2, 2, 0, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2,
         2, 2, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2,
         2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 1, 2, 2,
         2, 0, 0, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 0, 1, 0, 2, 2, 0, 1, 1, 2, 2, 2,
         2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2,
         0, 0, 2, 2, 2, 2, 2, 2], device='cuda:0'),
 'labels_2': tensor([ 7,  7,  7,  7,  7,  7,  7,  7,  7,  7, 14,  7,  7,  7, 14,  7,  7,  7,
          7,  7,  7,  7,  6,  7,  7,  7,  7, 10,  7,  7,  7,  7, 14,  7,  7,  7,
          7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7, 14, 10,  7,
          6,  7, 14,  7,  7,  7,  7,  7,  7,  7, 10,  7,  7,  7,  7,  7,  7,  7,
          7,  7,  7,  7,  7,  7,  7,  7,  7,  7, 14, 10,  7,  7,  7,  7,  7,  7,
          7,  7,  7,  6,  7,  7,  7,  7,  7,  7,  7,  7,  7, 10,  7,  7,  7,  7,
          7,  7,  7,  7,  7,  7, 

In [ ]:
from utils.metrics import get_multilevel_metrics
get_multilevel_metrics(preds,batch, cfg)

{'labels_0': {'overall': {'overall_accuracy': 0.109375,
   'confusion_matrix': array([[ 0.,  0.,  0.],
          [15., 14., 99.],
          [ 0.,  0.,  0.]], dtype=float32)},
  'classwise': {'iou': array([0.      , 0.109375, 0.      ], dtype=float32),
   'precision': array([0.        , 0.99999994, 0.        ], dtype=float32),
   'recall': array([0.      , 0.109375, 0.      ], dtype=float32),
   'f1_score': array([0.        , 0.19718291, 0.        ], dtype=float32),
   'class_acc': array([0.      , 0.109375, 0.      ], dtype=float32)}},
 'labels_2': {'overall': {'overall_accuracy': 0.0,
   'confusion_matrix': array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
            0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
            0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  2., 55.,  0.,  0.,  3.,  0.,  0.,
            0.,  4.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  2., 43

In [ ]:
# Training       : 100%|██████████| 1804/1804 [19:05<00:00,  1.57it/s]
# Validation     : 100%|██████████| 835/835 [07:57<00:00,  1.75it/s]
# Epoch-000: Lt:0.4091 Lv:1.4503


# Training       : 100%|██████████| 1804/1804 [19:28<00:00,  1.54it/s]
# Validation     : 100%|██████████| 835/835 [07:31<00:00,  1.85it/s]
# Epoch-001: Lt:0.3276 Lv:1.1352


# Training       : 100%|██████████| 1804/1804 [19:20<00:00,  1.55it/s]
# Validation     : 100%|██████████| 835/835 [07:06<00:00,  1.96it/s]
# Epoch-002: Lt:0.2901 Lv:1.0932


# Training       : 100%|██████████| 1804/1804 [19:16<00:00,  1.56it/s]
# Validation     : 100%|██████████| 835/835 [06:57<00:00,  2.00it/s]
# Epoch-003: Lt:0.2891 Lv:0.7242




In [ ]:
# visualize normals
# proj = train_ds.projects[7]
for i, proj in enumerate(train_ds.projects):
    PlyData([PlyElement.describe(pd.DataFrame(np.concatenate([proj['xyz'],proj['normals'],np.squeeze(proj['incAngles'])[:,None],np.proj['distanceFromScanner'][:,None], proj['sop_ids'][:,None]], axis=1),columns=['x','y','z','nx','ny','nz','incAngle','distanceFromScanner','sop_ids']).to_records(index=False),'vertex')]).write(f"./_temp/proj_{i:03}.ply")

In [ ]:
for arr in [proj['xyz'],proj['normals'],np.squeeze(proj['incAngles'])[:,None],proj['distanceFromScanner'][:,None], proj['sop_ids'][:,None]]:
    print(arr.shape)

In [ ]:
np.set_printoptions(suppress=True)
train_ds[0]['features_neibors'].std(axis=0)

In [ ]:
np.set_printoptions(suppress=True)
train_ds[0]['features_neibors'].std(axis=0)

In [ ]:
proj['incAngles'].mean()

In [ ]:
proj['incAngles'].std()

In [ ]:
proj['distanceFromScanner'].mean()

In [ ]:
(proj['distanceFromScanner']-proj['distanceFromScanner'].mean()).std()